In [ ]:
import os
from pathlib import Path
from sqlalchemy import create_engine

import xmlrpc.client
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

api_url = os.environ.get('ODOO_URL_API')
api_db = os.environ.get('ODOO_DB_API')
api_username = os.environ.get('ODOO_USERNAME_API')
api_clave = os.environ.get('ODOO_CLAVE_API')

common = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/common')
uid = common.authenticate(api_db, api_username, api_clave, {})
models = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/object')

db_file = 'comisiones.db'
db_file_path_str = str(Path().cwd().parent.parent.joinpath(f'data/{db_file}'))

engine = create_engine(f'sqlite:///{db_file_path_str}')

In [ ]:
with engine.connect() as conn, conn.begin():  
    ventas_enero = pd.read_sql_table('ventas_enero', conn, dtype_backend='numpy_nullable')
    ventas_febrero = pd.read_sql_table('ventas_febrero', conn, dtype_backend='numpy_nullable')
    ventas_marzo = pd.read_sql_table('ventas_marzo', conn, dtype_backend='numpy_nullable')
    ventas_abril = pd.read_sql_table('ventas_abril', conn, dtype_backend='numpy_nullable')
    ventas_mayo = pd.read_sql_table('ventas_mayo', conn, dtype_backend='numpy_nullable')
    ventas_junio = pd.read_sql_table('ventas_junio', conn, dtype_backend='numpy_nullable')
    ventas_año = pd.read_sql_table('ventas_año', conn, dtype_backend='numpy_nullable')

engine.dispose()

In [76]:
db_file1 = 'proveedores_oficiales.xlsx'
db_file1_path_str = str(Path().cwd().parent.parent.joinpath(f'data/compras/{db_file1}'))

proveedores = pd.read_excel(db_file1_path_str, dtype_backend='numpy_nullable')
prov_oficiales = proveedores.loc[proveedores['oficial'] == 1][['partner_id', 'partner_name']]
prov_locales = proveedores.loc[proveedores['oficial'] == 0][['partner_id', 'partner_name']]

In [77]:
db_file2 = 'productos_sin_compra.xlsx'
db_file2_path_str = str(Path().cwd().parent.parent.joinpath(f'data/compras/{db_file2}'))

productos_sin_compra = pd.read_excel(db_file2_path_str, dtype_backend='numpy_nullable')

In [ ]:
fields_compras_doc = ['name', 'state','partner_id', 'partner_ref', 'date_approve', 'x_fecha_factura', 'user_id', 'create_uid']

ids_compras_doc = models.execute_kw(api_db, uid, api_clave, 'purchase.order', 'search', [[("state", "in", ("purchase", "done"))]])
json_compras_doc = models.execute_kw(api_db, uid, api_clave, 'purchase.order', 'read', [ids_compras_doc], {'fields': fields_compras_doc})

In [ ]:
data_compras_doc = []

for compra in json_compras_doc:
    new = {}
    new['order_id'] = compra['id']
    new['order_name'] = compra['name']
    new['order_state'] = compra['state']
    new['order_date'] = compra['date_approve'] if compra['date_approve'] else pd.NA
    new['partner_id'] = compra['partner_id'][0]
    new['partner_name'] = compra['partner_id'][1]
    new['partner_fact_ref'] = compra['partner_ref']
    new['partner_fact_date'] = compra['x_fecha_factura'] if compra['x_fecha_factura'] else pd.NA
    new['capturista'] = compra['create_uid'][1] if compra['create_uid'] else pd.NA
    new['vendedora'] = compra['user_id'][1] if compra['user_id'] else pd.NA

    data_compras_doc.append(new)

compras_doc = pd.DataFrame(data_compras_doc)
compras_doc['order_date'] = pd.to_datetime(compras_doc['order_date'], format='%Y-%m-%d %H:%M:%S')
compras_doc['partner_fact_date'] = pd.to_datetime(compras_doc['partner_fact_date'], format='%Y-%m-%d')

In [ ]:
fields_compras_line = ['order_id', 'date_approve', 'partner_id','product_id', 'product_qty', 'price_unit_discounted']

ids_compras_line = models.execute_kw(api_db, uid, api_clave, 'purchase.order.line', 'search', [[("order_id.id", "in", ids_compras_doc)]])
json_compras_line = models.execute_kw(api_db, uid, api_clave, 'purchase.order.line', 'read', [ids_compras_line], {'fields': fields_compras_line})

In [ ]:
data_compras_line = []

for line in json_compras_line:
    new = {}
    new['line_id'] = line['id']
    new['order_id'] = line['order_id'][0]
    new['order_name'] = line['order_id'][1]
    new['order_date'] = line['date_approve'] if line['date_approve'] else pd.NA
    new['partner_id'] = line['partner_id'][0]
    new['partner_name'] = line['partner_id'][1]
    new['product_id_pp'] = line['product_id'][0]
    new['product_name'] = line['product_id'][1]
    new['product_qty'] = line['product_qty']
    new['product_price'] = line['price_unit_discounted']
    
    data_compras_line.append(new)

compras_linea = pd.DataFrame(data_compras_line)
compras_linea['order_date'] = pd.to_datetime(compras_linea['order_date'], format='%Y-%m-%d %H:%M:%S')

compras_linea['oficial'] = compras_linea['partner_id'].isin(prov_oficiales['partner_id'])

In [99]:
compras = (pd.concat
                ([
                pd.merge(compras_linea,
                   compras_doc[['order_id', 'partner_fact_ref', 'partner_fact_date', 'capturista', 'vendedora']], 
                    how='left', 
                    on='order_id'),
                productos_sin_compra
               ])
        )


cols_to_Int64 = ['line_id', 'order_id', 'partner_id', 'product_id_pp']
compras[cols_to_Int64] = compras[cols_to_Int64].astype('Int64')

compras['tolerance_order_date'] = compras['order_date'] + pd.Timedelta(days=5)

In [91]:
# Línea para comprobrar que el 100% de los proveedores de Odoo están calificados en la lista de proveedores oficiales

check1 = (compras_doc[~compras_doc['partner_id'].isin(proveedores['partner_id'])]).drop_duplicates('partner_id')
not check1.empty and print('Hay proveedores no calificados')
check1


,order_id,order_name,order_state,order_date,partner_id,partner_name,partner_fact_ref,partner_fact_date,capturista,vendedora


# Pruebas

In [95]:
ventas_año['costo_producto'] = pd.NA
ventas_año['costo_order_date'] = pd.NaT
ventas_año['costo_order_line_id'] = pd.NA

for i in range(len(ventas_año)):

    minidf = (compras[
            (compras['order_date'] <= pd.to_datetime(ventas_año['invoice_date'].iloc[i]) + pd.Timedelta(days=5)) 
            & (compras['product_id_pp'] == ventas_año['product_id'].iloc[i])]
            .sort_values('order_date', ascending = False)
            )

    if not minidf.empty:
        ventas_año.iloc[i,30] = minidf['product_price'].iloc[0]
        ventas_año.iloc[i,31] = minidf['order_date'].iloc[0]
        ventas_año.iloc[i,32] = minidf['line_id'].iloc[0]

    else:
        minidf_costo_inicial = productos_sin_compra[productos_sin_compra['product_id_pp'] == ventas_año['product_id'].iloc[i]]

        if not minidf_costo_inicial.empty:
            ventas_año.iloc[i,30] = minidf_costo_inicial['product_price'].iloc[0]
            ventas_año.iloc[i,31] = minidf_costo_inicial['order_date'].iloc[0]

In [120]:
wep = (
    pd.merge_asof(
        left = ventas_año[['fact_doc_id', 'invoice_date', 'product_id']].sort_values('invoice_date'),
        right = compras[['product_id_pp', 'tolerance_order_date', 'product_price']].sort_values('tolerance_order_date'), 
        
        left_by = 'product_id', 
        right_by = 'product_id_pp', 
        
        left_on = 'invoice_date', 
        right_on = 'tolerance_order_date', 

        # tolerance = pd.Timedelta(days=1),
        
        direction = 'backward')
)

wep

,fact_doc_id,invoice_date,product_id,product_id_pp,tolerance_order_date,product_price
0,114,2024-01-02,8068,8068,2023-11-15 00:00:00,4.17
1,369,2024-01-02,8954,8954,2023-12-28 00:00:00,15.52
2,372,2024-01-02,8620,8620,2023-12-14 00:00:00,10.90
3,372,2024-01-02,9261,9261,2023-12-14 00:00:00,19.54
4,376,2024-01-02,9507,9507,2023-12-12 00:00:00,22.89
...,...,...,...,...,...,...
62756,107789,2024-06-24,10335,10335,2024-04-15 19:44:33,39.14
62757,107789,2024-06-24,13073,13073,2024-02-05 16:57:49,208.31
62758,107785,2024-06-24,9389,9389,2024-06-15 21:04:41,21.41
62759,107795,2024-06-24,8059,8059,2024-05-13 18:25:58,4.10


In [128]:
product_id_to_check = 8059

wep.loc[wep['product_id'] == product_id_to_check]

,fact_doc_id,invoice_date,product_id,product_id_pp,tolerance_order_date,product_price,wep
251,742,2024-01-03,8059,8059,2023-11-23 00:00:00,4.10,True
6501,10045,2024-01-20,8059,8059,2023-11-23 00:00:00,4.10,True
6581,10165,2024-01-20,8059,8059,2023-11-23 00:00:00,4.10,True
7830,12056,2024-01-24,8059,8059,2023-11-23 00:00:00,4.10,True
11011,18154,2024-02-02,8059,8059,2023-11-23 00:00:00,4.10,True
11271,17915,2024-02-02,8059,8059,2023-11-23 00:00:00,4.10,True
12400,19906,2024-02-06,8059,8059,2023-11-23 00:00:00,4.10,True
14227,22954,2024-02-10,8059,8059,2023-11-23 00:00:00,4.10,True
14696,24413,2024-02-13,8059,8059,2023-11-23 00:00:00,4.10,True
17311,27943,2024-02-20,8059,8059,2023-11-23 00:00:00,4.10,True


In [127]:
compras.loc[compras['product_id_pp'] == product_id_to_check]

,line_id,order_id,order_name,order_date,partner_id,partner_name,product_id_pp,product_name,product_qty,product_price,oficial,partner_fact_ref,partner_fact_date,capturista,vendedora,tolerance_order_date
3596,12549,2120,P02107 (AA 1062894),2024-05-08 18:25:58,5283,Chapa Industrias,8059,"[2157] Escuadra Reforzada Union 2.5x2.5"" #1880...",240.00,4.10,True,AA 1062894,2024-05-06,Elsa Ivette Diaz Leyva,Mariana Araceli Carvajal Flores,2024-05-13 18:25:58
4854,10621,1792,P01779 (AA 1059173),2024-04-19 21:52:21,5283,Chapa Industrias,8059,"[2157] Escuadra Reforzada Union 2.5x2.5"" #1880...",720.00,4.10,True,AA 1059173,NaT,Alexa Yadira Mazariegos Zunun,Rosario Martinez Zarate,2024-04-24 21:52:21
4888,10544,1788,P01775 (AA 1059299),2024-04-19 20:48:44,5283,Chapa Industrias,8059,"[2157] Escuadra Reforzada Union 2.5x2.5"" #1880...",240.00,4.10,True,AA 1059299,2024-04-17,Alexa Yadira Mazariegos Zunun,Mariana Araceli Carvajal Flores,2024-04-24 20:48:44
604,<NA>,<NA>,SAE,2023-11-18 00:00:00,<NA>,NaN,8059,"Escuadra Reforzada Union 2.5x2.5"" #1880 *2157*",NaN,4.10,NaN,NaN,NaT,NaN,NaN,2023-11-23 00:00:00


In [ ]:
# ventas_año[ventas_año['costo_producto'].isna()].drop_duplicates('product_id').to_excel('wep.xlsx')
ventas_año[ventas_año['costo_producto'].isna()].drop_duplicates('product_id')[['product_id', 'product_name']]

In [ ]:
#Borrar esta línea, es provisional. Se tienen que corregir usando pd.NaN

for i in range(len(ventas_año['price_subtotal'])):
    ventas_año['price_subtotal'].iloc[i] = 0.01 if ventas_año['price_subtotal'].iloc[i] == 0 else ventas_año['price_subtotal'].iloc[i]

In [ ]:
ventas_año['costo_date_dif'] = ((ventas_año['invoice_date'] - ventas_año['costo_order_date']).dt.days).astype('Int64')
ventas_año['costo_subtotal'] = ventas_año['quantity'] * ventas_año['costo_producto']
ventas_año['utilidad_subtotal'] = ventas_año['price_subtotal'] - ventas_año['costo_subtotal']
ventas_año['utilidad_%'] = ((ventas_año['price_subtotal'] / ventas_año['costo_subtotal']) - 1) * 100
ventas_año['marg_util_%'] = (ventas_año['utilidad_subtotal'] / ventas_año['price_subtotal']) * 100


cols_ventas = ['fact_doc_id', 'name', 'invoice_date', 'partner_id',
       'partner_name', 'salesperson_id', 'salesperson_name', 'sale_team_description', 'business_model',
       'product_id', 'product_name', 'quantity', 'price_subtotal',
       'costo_subtotal', 'costo_order_date', 'costo_order_line_id',
       'costo_date_dif', 'utilidad_subtotal', 'utilidad_%', 'marg_util_%', 'costo_producto']

ventas = ventas_año[cols_ventas]

In [112]:
prod_varios_prov = compras[['partner_id', 'partner_name', 'product_id_pp', 'product_name']].groupby('product_id_pp').agg({'product_name': ['first'], 'partner_id': ['mean', 'first']})
prod_varios_prov.columns = ['product_name', 'mean', 'first']
prod_varios_prov['diff'] = prod_varios_prov['mean']  == prod_varios_prov['first']
prod_varios_prov.loc[prod_varios_prov['diff'] == False]

,product_name,mean,first,diff
product_id_pp,,,,
7480,"[2538] Rondana Plana 1/4"" C#181 *2538*","5,337.00",5366,False
7481,[2540] Rondana Plana 3/16 C#184 *2540*,"5,293.50",5366,False
7483,"[6528] Pija Autorr. C/Rondana 8""*1/2"" Galv. *6...","5,345.29",5366,False
7510,"[5388] Tuerca Hexagonal 1/4"" Hncz (5600/Ct) C...","5,329.75",5366,False
7511,"[10179] Taq. Plastico El-Pro 1/4"" Blanco TQ-01...","5,328.20",5313,False
...,...,...,...,...
29236,[4413] Brida Flexible Coflex PB-300 Larga *4413*,"5,326.33",5335,False
29241,[4426] Color Cemento Negro *4426*,"5,236.50",5143,False
29255,[4497] Guantes De Corte De Nitrilo 48-22-8902 ...,"5,179.80",5128,False


In [116]:
prod_varios_prov.loc[prod_varios_prov['diff'] == False].reset_index()['product_id_pp']

0       7480
1       7481
2       7483
3       7510
4       7511
       ...  
204    29236
205    29241
206    29255
207    29436
208    29452
Name: product_id_pp, Length: 209, dtype: Int64

In [118]:
compras[compras['product_id_pp'].isin(prod_varios_prov.loc[prod_varios_prov['diff'] == False].reset_index()['product_id_pp'])]

,line_id,order_id,order_name,order_date,partner_id,partner_name,product_id_pp,product_name,product_qty,product_price,oficial,partner_fact_ref,partner_fact_date,capturista,vendedora,tolerance_order_date
40,19046,3099,P03085 (MXF 759367),2024-06-25 00:51:27,5244,Makita Mexico,14016,[8635] Maletin D/Broca SDS-Plus MK D-31734 13p...,2.00,666.00,True,MXF 759367,2024-06-20,Alexa Yadira Mazariegos Zunun,Mariana Araceli Carvajal Flores,2024-06-30 00:51:27
49,18959,3093,P03079 (MXF 759368),2024-06-24 21:42:34,5244,Makita Mexico,14423,"[MAK9554HNG] Esmeriladora Ang 4 1/2"" 710W 9554...",5.00,943.00,True,MXF 759368,2024-06-20,Alexa Yadira Mazariegos Zunun,Mariana Araceli Carvajal Flores,2024-06-29 21:42:34
116,18837,3089,P03075 (AA 1072328),2024-06-24 21:08:30,5283,Chapa Industrias,9169,[13374] Bis. Bidi. Brazo Corto #3655 (Par) *13...,250.00,19.25,True,AA 1072328,2024-06-24,Alexa Yadira Mazariegos Zunun,Rosario Martinez Zarate,2024-06-29 21:08:30
123,18844,3089,P03075 (AA 1072328),2024-06-24 21:08:30,5283,Chapa Industrias,11326,"[15112] Pija Drywall 8 x 2"" 15422 (Kg) *15112*",50.00,70.63,True,AA 1072328,2024-06-24,Alexa Yadira Mazariegos Zunun,Rosario Martinez Zarate,2024-06-29 21:08:30
145,18777,3082,P03068 (SJCFCO 16222),2024-06-24 15:43:49,5326,Madereria El Pino De Los Cabos,12827,[13712] Barrote 2 * 4 * 12 Americano *13712*,25.00,181.04,False,SJCFCO 16222,2024-06-22,Alexa Yadira Mazariegos Zunun,Yamilet Blanco Salas,2024-06-29 15:43:49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5614,<NA>,<NA>,SAE,2023-10-16 00:00:00,<NA>,NaN,14016,Maletin D/Broca SDS-Plus MK D-31734 13pz *8635*,NaN,599.00,NaN,NaN,NaT,NaN,NaN,2023-10-21 00:00:00
6085,<NA>,<NA>,SAE,2023-11-23 00:00:00,<NA>,NaN,14671,Malla Sombra 2x100m 90% Beige Raschel *96679*,NaN,"3,620.70",NaN,NaN,NaT,NaN,NaN,2023-11-28 00:00:00
6768,<NA>,<NA>,SAE,2023-01-19 00:00:00,<NA>,NaN,27422,Silicon Cafe DAP Alex Plus 300 ml *2468*,NaN,82.76,NaN,NaN,NaT,NaN,NaN,2023-01-24 00:00:00
7262,<NA>,<NA>,SAE,2023-11-08 00:00:00,<NA>,NaN,28253,Detector Tipo Lapiz 1AC-A1-II Mca Fluke *11508*,NaN,828.25,NaN,NaN,NaT,NaN,NaN,2023-11-13 00:00:00


In [ ]:
product_id_to_check = 12827

# compras.loc[compras['product_id_pp'] == product_id_to_check].to_excel('compras.xlsx')
# ventas.loc[ventas['product_id'] == product_id_to_check].to_excel('ventas.xlsx')